# Modeling Local Demographics based on Local Venues in Toronto, Canada

*By Andrew Dahlstrom*

*10/04/2019*

### Introduction

This Notebook is my submission for the final project in the IBM Applied Data Science Capstone Course. This assignment requires that I design an imagined scenario which will allow me to demonstrate the techniques and methodologies I have learned in the IBM Applied Data Science Specialization program as well as utilize the FourSquare API and the scikit-learn library for learning algorithms in Python. The scenario I devised for this project is that I have been contracted by International Health Organization to conduct an exploratory case with the goal to improve methodologies for building demographic maps. To accomplish this goal, I have chosen to experiment with modeling and predicting local demographics based on local venue data for neighborhoods in the city of Toronto, Canada. 

### Background

Population demographic maps are useful tools for many humanitarian efforts including to manage disease outbreaks, water scarcity, disaster relief efforts, electrical grid expansion, expansion of health or education services etc. Demographic data is not always readily available in some areas so any contribution to methodology that can improve the accuracy of population maps could be useful to humanitarian efforts. The goal of this project is to explore how accurately local venue data can predict the demographic data for that neighborhood (organized by postal code) using relevant machine learning techniques to create a prediction model. 

### Data

This city of Toronto was selected because of the detailed and recent demographic data available publicly and the large collection of venue data available for each neighborhood. The data for this project has been collected from the following sources:

* Toronto postal code data can be found on [Wikipedia](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M).
* The demographic data for Toronto has been collected from the [2016 Census](https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/pd-pl/Table.cfm?Lang=Eng&T=1201&S=22&O=A).
* The local venue data will be retrieved from the [FourSquare API](https://developer.foursquare.com/).
* Latitude and longitude geospatial data for postal codes provided by IBM course website

#### Construct Dataframes
The first step is to create a dataframe of postal codes for the city of Toronto using a web scraper in order to organize the venue and demographic data into smaller communities.

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd

In [2]:
# Scrape text from wikitable online and load into a Pandas data frame

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
#print(soup.prettify())

for table in soup.find_all('table', class_= 'wikitable'):
    neightable = []
    
    for row in table.find_all('tr'):
        neighrow = []
        
        for data in row.find_all('td'):
            neighrow.append(data.text.rstrip('\n'))
        
        neightable.append(neighrow)

#Clean data remove unassigned boroughs

neighdf = pd.DataFrame(neightable, columns = ['Postal Code', 'Borough', 'Neighborhood'])
neighdf.drop(index=0, inplace=True)
todrop = neighdf[neighdf['Borough'] == "Not assigned"].index
neighdf.drop(todrop, inplace=True)
neighdf.reset_index(drop=True).head(5)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In order to clean the Toronto postal code data we need to merge neighborhoods that share the same postal code and give unnamed neighbourhoods the name of their borough.

In [3]:
# Replace unassigned neighborhoods or neighborhoods with incorrect names with their borough name 

for index, row in neighdf.iterrows(): 
    s = neighdf.at[index, 'Neighborhood']
    if  s == "Not assigned" or len(s.split()) > 4 :
        neighdf.at[index, 'Neighborhood'] = neighdf.at[index, 'Borough']

# Merge neighborhoods with same postalcode

neighdf = neighdf.groupby(['Postal Code','Borough'])['Neighborhood'].apply(', '.join).reset_index()
neighdf.head(5)       

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Next we need to get the csv file from the link provided in the course website to get the latitude and longitude coordinates for each postal code.

In [4]:
import io
geourl="http://cocl.us/Geospatial_data"
s = requests.get(geourl).content
geodata = pd.read_csv(io.StringIO(s.decode('utf-8')))
geodata.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Add latitude and longitude data to neighborhood dataframe.

In [5]:
for index, row in neighdf.iterrows():   
    for i, r in geodata.iterrows():
        if neighdf.at[index, 'Postal Code'] == geodata.at[i, 'Postal Code']:
            neighdf.at[index, 'Latitude'] = geodata.at[i, 'Latitude']
            neighdf.at[index, 'Longitude'] = geodata.at[i, 'Longitude']

neighdf.shape

(103, 5)

Next we need the FourSquare API to get the venue data for each postal code.

In [6]:
# The code was removed by Watson Studio for sharing.

In [30]:
### Create dataframe containing venue data for each postal code ###

# Define function to get venue data for each postal code.
# Takes as an argument location name, latitude and logitude coordinates
# returns a dataframe containing venue data for the nearest 
# LIMIT number of venues within the radius

# Max number of venues within radius
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=8000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Postal Code', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
# Create the dataframe using the getNearbyVenues function

venues = getNearbyVenues(names=neighdf['Postal Code'],
                                 latitudes=neighdf['Latitude'],
                                 longitudes=neighdf['Longitude'],
                                )

venues.shape

(10300, 7)

In [32]:
print('There are {} unique venue categories and the following venue counts for each postal code...'.format(len(venues['Venue Category'].unique())))
venues.groupby('Postal Code').count().head(5)

There are 219 unique venue categories and the following venue counts for each postal code...


,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,
M1B,100,100,100,100,100,100
M1C,100,100,100,100,100,100
M1E,100,100,100,100,100,100
M1G,100,100,100,100,100,100
M1H,100,100,100,100,100,100


Now that we have the venue data, the next step is to build the dataframe of community venue profiles for each postal code based on the venue data. I will use a method called one hot encoding to build a binary classification table which can then be used to build a frequency table for the occurences of a venue in each category for each postal code. 

In [39]:
# One hot encoding
toronto_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# Add back in postal code column and move it to front
toronto_onehot['Postal Code'] = venues['Postal Code']
fixed_columns = list(toronto_onehot)
fixed_columns.insert(1, fixed_columns.pop(
    fixed_columns.index('Postal Code')))
toronto_onehot = toronto_onehot.loc[:, fixed_columns]

# Build frequency table by postal code and by taking the mean of the frequency of occurrence for each category
toronto_venues = toronto_onehot.groupby('Postal Code').mean().reset_index()
toronto_venues.head(5)

,Postal Code,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,M1B,0.0,0.0,0.0,0.00,0.0,0.0,0.01,0.01,0.01,...,0.0,0.00,0.01,0.0,0.0,0.01,0.0,0.0,0.02,0.10
1,M1C,0.0,0.0,0.0,0.01,0.0,0.0,0.00,0.00,0.01,...,0.0,0.00,0.00,0.0,0.0,0.01,0.0,0.0,0.02,0.10
2,M1E,0.0,0.0,0.0,0.00,0.0,0.0,0.01,0.02,0.01,...,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.02,0.09
3,M1G,0.0,0.0,0.0,0.00,0.0,0.0,0.01,0.02,0.01,...,0.0,0.01,0.01,0.0,0.0,0.00,0.0,0.0,0.02,0.05
4,M1H,0.0,0.0,0.0,0.00,0.0,0.0,0.01,0.02,0.01,...,0.0,0.01,0.01,0.0,0.0,0.00,0.0,0.0,0.02,0.01


Now that the neighborhood venue profile dataframe is complete, the next step is to build the neighborhood demographics profile dataframe using data from the 2016 Canada Census.

In [40]:
from zipfile import ZipFile

# Get demographic data from Canada Census website, load into dataframe
url = "https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/dt-td/CompDataDownload.cfm?LANG=E&PID=109790&OFT=CSV"
r = requests.get(url)
z = ZipFile(io.BytesIO(r.content))
z.extractall()

demo = pd.read_csv(z.open('98-400-X2016008_English_CSV_data.csv'), 
                     usecols = ['L9Z', 'Average age', '50.0'],
                     sep = ',', skiprows = 112395, nrows = 12192)
demo.rename(columns = {"L9Z": "Postal Code", "Average age": "Index", 
                       "50.0" : "Population"}, inplace = True)
demo.head()

,Postal Code,Index,Population
0,M1B,Total - Age,66110.0
1,M1B,0 to 14 years,11535.0
2,M1B,0 to 4 years,3540.0
3,M1B,Under 1 year,675.0
4,M1B,1,705.0


Next let's organize the table so that the index is postal codes and the columns are population per age groups in increments of 5 years.

In [41]:
toronto_demo = demo.pivot(index = 'Postal Code', columns = 'Index', values = 'Population')
keep_columns = ['0 to 4 years', '5 to 9 years', '10 to 14 years', '15 to 19 years', 
                '20 to 24 years', '25 to 29 years', '30 to 34 years', '35 to 39 years',
                '40 to 44 years', '45 to 49 years', '50 to 54 years', '55 to 59 years',
                '60 to 64 years', '65 to 69 years', '70 to 74 years', '75 to 79 years',
                '80 to 84 years', '85 to 89 years', '90 to 94 years', '95 to 99 years',
                '100 years and over']
toronto_demo = toronto_demo[keep_columns].reset_index()
toronto_demo.head()

Index,Postal Code,0 to 4 years,5 to 9 years,10 to 14 years,15 to 19 years,20 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,40 to 44 years,...,55 to 59 years,60 to 64 years,65 to 69 years,70 to 74 years,75 to 79 years,80 to 84 years,85 to 89 years,90 to 94 years,95 to 99 years,100 years and over
0,M1B,3540.0,3920.0,4080.0,4680.0,5090.0,4835.0,4415.0,3950.0,4150.0,...,4570.0,4070.0,3615.0,2470.0,1610.0,925.0,530.0,220.0,65.0,10.0
1,M1C,1575.0,1760.0,1905.0,2380.0,2695.0,2130.0,1870.0,1955.0,1965.0,...,2970.0,2670.0,2340.0,1695.0,1090.0,665.0,385.0,185.0,40.0,5.0
2,M1E,2405.0,2585.0,2585.0,3100.0,3450.0,2975.0,2665.0,2550.0,2720.0,...,3610.0,3050.0,2370.0,1840.0,1545.0,1195.0,840.0,405.0,105.0,15.0
3,M1G,1720.0,1925.0,1950.0,2140.0,2350.0,2125.0,1835.0,1810.0,1775.0,...,1970.0,1530.0,1320.0,1015.0,925.0,685.0,370.0,140.0,20.0,5.0
4,M1H,1330.0,1365.0,1175.0,1320.0,1970.0,2000.0,1910.0,1705.0,1540.0,...,1605.0,1340.0,1010.0,820.0,670.0,655.0,385.0,185.0,35.0,5.0


#### Notes about the data: 

* The demographic data comes from a 2016 census but the venue data is current meaning that the demographic data is lagging behind the venue data by a couple of years so this will affect the accuracy of the model.
* For financial reasons I must limit the number of venues I am able to collect data on for each neighborhood. I will therefore find the distribution of the venues for each category within each neighborhood in order to create a venue profile for each neighborhood.  
* I will be exploring how well venue data predicts the age distribution of the population in each neighborhood separated into 5 year increments.

### Methodology
Now it's time for the exciting part! First I will test different machine learning regression algorithms from the sklearn package to fit the venue data then analyze which model is best at predicting neighborhood age demographics. Second, I will take a different approach to understanding the relationship between dataframes by utilizing clustering techniques to look for trends between venues and age demographics. 

#### Preprocess Data
First we can normalize our data by finding the mean for each category and then subtracting that from the mean of each postal code. This will make it easier to notice differences between the neighborhoods. We can also use sklearn built in preprocessing to normalize our labels and features with the StandardScaler.

In [42]:
from sklearn.preprocessing import StandardScaler
features_scaler = StandardScaler()

features = toronto_venues

# Drop the rows in the features set that we do not have population data for
rowsDrop = list(set(toronto_venues['Postal Code']).symmetric_difference(set(toronto_demo['Postal Code'])))
features.drop(features.loc[features['Postal Code'].isin(rowsDrop)].index, inplace=True)

# Drop Postal Code column to have all numeric
features.drop(['Postal Code'], axis = 1, inplace = True)
features.shape

(96, 219)

In [43]:
labels_scaler = StandardScaler()

labels = toronto_demo
labels.drop(['Postal Code'], axis = 1, inplace = True)
labels = toronto_demo.apply(lambda x: x / x.sum(), axis = 1)
labels.head()

Index,0 to 4 years,5 to 9 years,10 to 14 years,15 to 19 years,20 to 24 years,25 to 29 years,30 to 34 years,35 to 39 years,40 to 44 years,45 to 49 years,...,55 to 59 years,60 to 64 years,65 to 69 years,70 to 74 years,75 to 79 years,80 to 84 years,85 to 89 years,90 to 94 years,95 to 99 years,100 years and over
0,0.053547,0.059295,0.061715,0.070791,0.076993,0.073136,0.066783,0.059749,0.062774,0.066707,...,0.069127,0.061564,0.054682,0.037362,0.024353,0.013992,0.008017,0.003328,0.000983,0.000151
1,0.044204,0.049397,0.053466,0.066798,0.075639,0.059781,0.052484,0.054869,0.055150,0.067078,...,0.083357,0.074937,0.065675,0.047572,0.030592,0.018664,0.010806,0.005192,0.001123,0.000140
2,0.051219,0.055053,0.055053,0.066021,0.073475,0.063359,0.056756,0.054307,0.057928,0.069002,...,0.076882,0.064956,0.050474,0.039186,0.032904,0.025450,0.017889,0.008625,0.002236,0.000319
3,0.057932,0.064837,0.065679,0.072078,0.079151,0.071573,0.061805,0.060963,0.059784,0.065173,...,0.066352,0.051533,0.044459,0.034187,0.031155,0.023072,0.012462,0.004715,0.000674,0.000168
4,0.054508,0.055943,0.048156,0.054098,0.080738,0.081967,0.078279,0.069877,0.063115,0.068033,...,0.065779,0.054918,0.041393,0.033607,0.027459,0.026844,0.015779,0.007582,0.001434,0.000205


In [44]:
norm_features = features_scaler.fit_transform(features)
norm_labels = labels_scaler.fit_transform(labels)

#### Split the Data into Training and Testing Sets:

In [45]:
from sklearn.model_selection import train_test_split

#Split the training data into a training and testing set
#Note the switch between features and labels
X_train, X_test, y_train, y_test = train_test_split(norm_features, norm_labels, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (76, 219) (76, 21)
Test set: (20, 219) (20, 21)


#### Find the best Regression Model :

In [50]:
from sklearn import linear_model
from sklearn import metrics
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.multioutput import MultiOutputRegressor
from sklearn import neighbors 

best_rscore = 0
best_model = 0
best_ev = 0

regressors = [
    #Linear regressors 
    #linear_model.LinearRegression(),
    #linear_model.SGDRegressor(),
    linear_model.BayesianRidge(),
    #linear_model.LassoLars(),
    #linear_model.PassiveAggressiveRegressor(),
    #linear_model.TheilSenRegressor(),
    
    #KNN regressors
    #neighbors.KNeighborsRegressor(),
    #neighbors.RadiusNeighborsRegressor()
    ]

for r in regressors:
    mr =  MultiOutputRegressor(r)
    mr.fit(X_train, y_train)
    yhat = mr.predict(X_test)
    this_rscore = r2_score(y_test, yhat, multioutput='uniform_average')
    this_ev = explained_variance_score(y_test, yhat, multioutput = 'uniform_average')
    
    if (best_rscore < this_rscore) :
        best_model = mr
        best_rscore = this_rscore
        best_ev = this_ev

In [51]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor

for d in range(1, 5):

    regressors = [
        DecisionTreeRegressor(max_depth = d),
        ExtraTreesRegressor(max_depth = d, n_estimators = 100),
        RandomForestRegressor(max_depth = d, n_estimators = 100)
    ]
    #print('Max Depth =', d, '\n')
    
    for r in regressors:
        dtr = r
        mdtr = MultiOutputRegressor(r)
        mdtr.fit(X_train, y_train)
        yhat = mdtr.predict(X_test)
        this_rscore = r2_score(y_test, yhat, multioutput='uniform_average')
        this_ev = explained_variance_score(y_test, yhat, multioutput = 'uniform_average')
        
        if (best_rscore < this_rscore):
            best_model = mdtr
            best_rscore = this_rscore
            best_ev = this_ev

From the linear, decision tree and knn regressors, the following model predicted age demographics with the best r2_score and expected variance. 

In [53]:
print(best_model, '\n',  'Num Estimators: ', len(best_model.estimators_), '\n', 'R2 Score: ', best_rscore, '\n', 'Explained Variance Score: ', best_ev)

MultiOutputRegressor(estimator=BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False),
           n_jobs=None) 
 Num Estimators:  21 
 R2 Score:  0.4502172465213065 
 Explained Variance Score:  0.5293373878460669


#### Analysis

The two primary metrics I utilized to score and compare the performance of the estimators were explained variance and r2 score (coefficient of determination) because this problem is a regression problem and multilabel in nature. Therefore among the provided sklearn metrics, r2_score and explained_variance_score provide an objective measure of overall correlation. It is also important to note that I chose the "uniform_average" parameter for both metrics because the venue data which I was able to obtain only included  100 highest ranked venues for each neighborhood so with this lack of data weighting venue categories differently would not be appropriate. 

The best model in this experiment turned out to be a linear regression model known as Bayesian Ridge with 21 estimators (one for each label). The r squared score of the linear regressor measures how correlated the predicted demographic data is to the venue test data. The score of .46 implies that the demographic data is overall positively correlated to the 21 estimators and approximately 46% of the age demographics can be explained by the venue categories. The explained variance score measures how well the model explains the variance of the prediction compared to the variance of the training data so the score of approximately .53 indicates that a little over half of the variance in the prediction is explained by the variance in the training data. 

Let's examine one of the estimators for the age group 0-4 years to see the intercept as well as the coefficients associated with each venue category sorted in descending order. The table below shows the venue categories which increase at the greatest rate in response to a predicted incremental increase in the age group 0-4 years.

In [54]:
age_cat = 0
single_label_coef = pd.DataFrame(list(zip(features.columns.tolist(), best_model.estimators_[age_cat].coef_)), 
               columns =['Category', 'Coefficient']).sort_values('Coefficient', ascending=False) 

print('Model Intercept: ', best_model.estimators_[age_cat].intercept_, '\n', 'Model Coefficients: ','\n', single_label_coef)

Model Intercept:  0.04315462074507209 
 Model Coefficients:  
                           Category  Coefficient
69             Egyptian Restaurant     0.056051
50                      Comic Shop     0.046842
28                         Brewery     0.042698
103                  Historic Site     0.034372
7                 Asian Restaurant     0.031776
34                         Butcher     0.031137
114                 Ice Cream Shop     0.030241
53      Construction & Landscaping     0.029344
183                   Soccer Field     0.026890
71            Ethiopian Restaurant     0.026198
119            Japanese Restaurant     0.025837
106                   Hockey Arena     0.021961
42                  Chocolate Shop     0.021000
43                          Circus     0.020880
215                  Women's Store     0.020866
47                     Coffee Shop     0.020766
211                Warehouse Store     0.019024
80                     Flower Shop     0.018490
61                Departm

### Clustering 
Next we explore a different approach using clustering algorithms. The algorithm will cluster the neighborhoods based on their demographic data then I will compare the venue categories for the neighborhoods in each cluster in similar cluster based on their frequency and novelty.

In [55]:
from sklearn.cluster import AffinityPropagation
from sklearn import metrics
# Use labels because that is the most complete data set
# Using Affinity Prop because dataset is small and to estimate number of clusters
toronto_affinity = AffinityPropagation().fit(norm_labels)

cluster_centers_indices = toronto_affinity.cluster_centers_indices_
alabels = toronto_affinity.labels_

n_clusters = len(cluster_centers_indices)

print('Estimated number of clusters: %d' % n_clusters)

Estimated number of clusters: 14


I initially leveraged the Affinity Propagation clustering algorithm to predict the appropriate number of clusters because this is a convenient feature of this particular clustering algorithm and since the dataset it small. I will then use the more traditional KMeans clustering to form the clusters and the following metrics to score (max score = 1) the model. Homogeneity describes how well cluster contains only members of a single label. Completeness measures how well all members of a given label are assigned to the a particular cluster. V-measure represents the harmonic mean between homogeneity and completeness.

In [56]:
from sklearn.cluster import KMeans

# Advantages to using kmeans is that it scales better for larger datasets
# 14 clusters provides perfect score on metrics
# set number of clusters
kclusters = n_clusters

# cluster neighborhoods by age demographics
toronto_clusters = KMeans(n_clusters=kclusters, random_state=0).fit(norm_labels)
klabels = toronto_clusters.labels_

#Compare clustering models
print("Homogeneity: %0.3f" % metrics.homogeneity_score(alabels, klabels))
print("Completeness: %0.3f" % metrics.completeness_score(alabels, klabels))
print("V-measure: %0.3f" % metrics.v_measure_score(alabels, klabels))

Homogeneity: 0.809
Completeness: 0.779
V-measure: 0.794


In [57]:
demo_clusters = pd.DataFrame(data = norm_labels[0:,0:], columns=labels.columns)
demo_clusters['Labels'] = toronto_clusters.labels_
demo_clusters_sum = demo_clusters.groupby(by=['Labels']).sum()

#Show which age groups had greatest occurance above mean in each cluster
for label in range(demo_clusters_sum.shape[0]):
    print(demo_clusters_sum.iloc[label, :].sort_values(ascending = False).head(5), '\n')

Index
45 to 49 years    5.950862
40 to 44 years    3.985381
10 to 14 years    3.978630
5 to 9 years      3.623529
50 to 54 years    3.377334
Name: 0, dtype: float64 

Index
30 to 34 years    15.346218
35 to 39 years    13.681714
25 to 29 years    13.113996
40 to 44 years     3.929779
20 to 24 years    -0.619862
Name: 1, dtype: float64 

Index
70 to 74 years        6.865712
65 to 69 years        6.173570
95 to 99 years        5.106074
75 to 79 years        4.608758
100 years and over    4.375913
Name: 2, dtype: float64 

Index
40 to 44 years    17.549786
35 to 39 years    12.338174
0 to 4 years       8.876693
45 to 49 years     8.785409
50 to 54 years     3.395479
Name: 3, dtype: float64 

Index
20 to 24 years        7.258659
25 to 29 years        2.834215
100 years and over    1.560189
30 to 34 years        1.156770
95 to 99 years        1.135773
Name: 4, dtype: float64 

Index
40 to 44 years    4.332026
55 to 59 years    3.308082
0 to 4 years      3.149284
35 to 39 years    2.770255
5

In [58]:
venue_clusters = pd.DataFrame(data = norm_features[0:,0:], columns=features.columns)
venue_clusters['Labels'] = toronto_clusters.labels_
venue_clusters_sum = venue_clusters.groupby(by=['Labels']).sum()

# Show which venues occured greatest from the mean
#for label in range(demo_clusters_sum.shape[0]):
#    print(venue_clusters_sum.iloc[label, :].sort_values(ascending = False).head(5), '\n')

Clustering provides a novel approach to analyzing the relationship between labels and features. We can compare the most interesting labels and features for each cluster by displaying the ones which have the greatest occurance above their normalized mean. Let's examine the top three labels and features for each cluster to see what trends may be apparent.

In [60]:
for label in range(demo_clusters_sum.shape[0]):
    print((demo_clusters_sum.iloc[label, :]).sort_values(ascending = False).head(2),'\n', 
          venue_clusters_sum.iloc[label, venue_clusters_sum.columns != 'Labels'].sort_values(ascending = False).head(4),'\n')

Index
45 to 49 years    5.950862
40 to 44 years    3.985381
Name: 0, dtype: float64 
 Hotel Bar                9.541599
Comic Shop               4.628448
Indonesian Restaurant    4.378803
Soccer Field             4.217249
Name: 0, dtype: float64 

Index
30 to 34 years    15.346218
35 to 39 years    13.681714
Name: 1, dtype: float64 
 Record Shop              13.856406
Spanish Restaurant       13.639887
Harbor / Marina          13.344922
Performing Arts Venue    13.141043
Name: 1, dtype: float64 

Index
70 to 74 years    6.865712
65 to 69 years    6.173570
Name: 2, dtype: float64 
 Korean Restaurant     6.892195
Hookah Bar            6.565447
Persian Restaurant    6.565447
Ramen Restaurant      6.313863
Name: 2, dtype: float64 

Index
40 to 44 years    17.549786
35 to 39 years    12.338174
Name: 3, dtype: float64 
 Circus                 16.426303
Comic Shop             13.529311
American Restaurant    13.528950
Ice Cream Shop         12.557368
Name: 3, dtype: float64 

Index
20 to 24 y

### Results

Due to the small sample size of only 96 neighborhoods and the large number of demographic and venue category labels (21 and 220 respectively), it is difficult to train and test a machine learning model on such a dataset. It is also important to note that there are less multioutput regression models available to experiment with the scope of this study than say single output regression or a classification problem. Despite the deficits in the supply of data discussed previously, several of the models performed surprisingly well. 

The Bayesian Ridge linear regressor in particular was able to explain almost 50% of the local demographic data based on the local venue category data. Some of the most apparent relationships occured with the gym and athletic venues being more popular among the ages 20 - 34 and the BBQ Joint which reigns supreme between the ages 45 - 74 which is then gradually replaced by asian and mediterrainian restaurants. 

The clustering model seemed to group people who are close in age together the most which also revealed some relationships among particular venue categories. Interestingly, similar venue categories did not seem appear for the same age groups but which were in different clusters.

### Discussion

This experimental study provides lots of space for further discussion and exploration. It would of course be useful to include more cities and larger venue category datasets in a future study but even given the results from this small study some important observations can be made. 

The relationship between the demographic and venue data can be more nuanced. For example, when examining the Bayesian Ridge regression model the estimator coefficients for the age group 0-4 show Egyptian restaurant, comic shop and brewery listed as the three most positively impacted venue categories which may seem counterintuitive however it is important to remember that people in the youngest age groups likely live with their parents who would be the ones using these venues. The clustering model provided a novel perspective of the relationship by associating some new venue categories with different age groups such as hotel bars being associated with people in their fourties or Comedy Clubs being popular with people in their twenties. It also revealed similar trends observed by the regression model such as fitness and outdoor related venues more popular among younger adults as well as restaurants of various Asian cuisines tending to be more popular among older adults. The clustering model also supported the relationship of the comic shop and Egyptian restaurant (but not the brewery) to the youngest age groups shown in the linear regressor coefficients.

One distinct type of venue category that relates to many different age demographics is restaurants of various cuisines. Given how ethnically diverse the city of Toronto is, examining its historic inflows of different ethnic populations immigrating at different time periods could provide more insight into the relationship between specific types of restaurants and local age demographics. This type of relationship may not be consistent in other cities or cities in other countries. Consequently, even though different venue categories of restaurants are very prevalent, they may not generalize well and perhaps should be merged into broader categories or simply as restaurants in order to pursue a model that could be applied more generally. This same rationale could be used to combine other venue categories into broader venue categories thus significantly shrinking the total number of feature categories especially if the neighborhood sample size is small.

### Conclusion

The purpose of this case study was primarily exploratory and experimental with the goal to understand novel approaches to improve methodologies for building demographic maps. Several different machine learning techniques were utilized to explore the relationship between neighborhood age demographic data and venue category data in the city of Toronto. For this type of multioutput regression problem it seems that a linear regression model such as Bayesian Ridge may produce estimators with the highest prediction efficacy. The clustering approach provides novel insights as well as supporting findings from the regression model making the two approaches complementary for this type of problem.

While the regression and clustering approaches can be mutually beneficial for mondeling local demographics based on local venues, there are still unique combinations of venues that arise in response to cultural, ethnic and other unique factors. In order to achieve a more  generalized model for predicting age demographics, forming broader feature categories in order to create more generalized neighborhood profiles could contribute to the methodology.

*For questions of comments please follow up at adahlstromcg@gmail.com*